In [ ]:
import dill
dill.dump_session("EXP4.db")

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split



In [2]:
from keras.models import Sequential
from keras.layers import Conv1D, ConvLSTM2D, MaxPooling1D, Flatten, Dense, Reshape,TimeDistributed,Conv2D,BatchNormalization,MaxPooling2D,ConvLSTM1D,LSTM



In [3]:
import tensorflow as tf
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)


In [4]:
# Read CSV File
df = pd.read_csv(r"C:\Users\mujta\OneDrive\Documents\GitHub\F20PA-DriverDrowsinessDetection\Experiments\mergedEMYA_min_max_normEMYA.csv")

In [5]:
# Select frames at 1 fps
df = df.iloc[::10]


In [6]:
df.drop(columns=["Unnamed: 0","Part_ID"],inplace=True)

In [7]:
df

,Label,EAR,MAR,MOE,Angle
0,0,0.695510,0.616864,0.391060,0.531289
10,0,0.748416,0.650276,0.388583,0.601121
20,0,0.777219,0.650798,0.376921,0.538889
30,0,0.310910,0.590178,0.616396,0.637834
40,0,0.677256,0.632891,0.409526,0.579984
...,...,...,...,...,...
422950,10,0.173777,0.563920,0.663220,0.630042
422960,10,0.296172,0.674049,0.558488,0.585958
422970,10,0.440577,0.598430,0.399449,0.607305
422980,10,0.197943,0.708829,0.690436,0.600016


In [8]:
int(df.shape[0]/5)

8460

In [9]:

from sklearn.metrics import precision_recall_fscore_support
# Function to get precision, recall, and F1 scores
def getPerformanceMetrics(model,X_test,y_test):

    y_pred_proba = model.predict(X_test)
    y_pred = np.argmax(y_pred_proba, axis=1)
    
    # Compute precision, recall, and F1 score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(y_test.flatten(), y_pred, average=None)
    # Lists of precision , recall, and F1 scores
    p_list = []
    r_list = []
    f_list = []
    # Print precision, recall, and F1 score for each class
    for i in range(len(precision)):
        p_list.append(precision[i])
        r_list.append(recall[i])
        f_list.append(f1[i])
        print(f"Class {i}:")
        print(f"  Precision: {precision[i]}")
        print(f"  Recall: {recall[i]}")
        print(f"  F1 Score: {f1[i]}")
    # Print Average values
    print(f"Avg Precision:{np.average(p_list)}")
    print(f"Avg Recall:{np.average(r_list)}")
    print(f"Avg F1 Score:{np.average(f_list)}")


In [10]:
# Function to set parameters and train model.
# Returns model, and test sets.
def compileModel(data,timestep,test_size,num_epochs):
    # Separate labels
    y=data["Label"]
    data = data.drop(columns=['Label'])
    # Reshape according to timestep
    df_reshaped = np.array(data).reshape(int(data.shape[0]/timestep),timestep,3)
    y = np.array(y)
    y_reshaped  = []
    for i in range(0, len(y), timestep):
        y_reshaped.append([y[i]])
    # Create test and  train sets.
    X_train, X_test, y_train, y_test = train_test_split(df_reshaped, y_reshaped, test_size=test_size, random_state=42)

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train= np.array(y_train)
    y_test = np.array(y_test)
    # Set labels as  0,1,2
    for i in range(0,len(y_train)):
        y_train[i]=int(y_train[i]/5)
    for i in range(0,len(y_test)):
        y_test[i]=int(y_test[i]/5)
    # Define model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(timestep, 3)))
    model.add(MaxPooling1D(pool_size=2))

    model.add(LSTM(64, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(3, activation='softmax'))  

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=num_epochs, batch_size=32, validation_split=0.2)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy:", accuracy)
    getPerformanceMetrics(model,X_test,y_test)
    return model, X_test, y_test






## Increase timestep 5,10,20

In [11]:
modelA , X_testA, y_testA = compileModel(data=df,
                                         timestep=5,
                                         test_size=0.2,
                                         num_epochs=50)




Epoch 1/50


170/170 [==============================] - 3s 5ms/step - loss: 1.0810 - accuracy: 0.3993 - val_loss: 1.0610 - val_accuracy: 0.4069
Epoch 2/50
170/170 [==============================] - 1s 3ms/step - loss: 1.0563 - accuracy: 0.4265 - val_loss: 1.0540 - val_accuracy: 0.4306
Epoch 3/50
170/170 [==============================] - 0s 3ms/step - loss: 1.0446 - accuracy: 0.4355 - val_loss: 1.0393 - val_accuracy: 0.4284
Epoch 4/50
170/170 [==============================] - 1s 3ms/step - loss: 1.0397 - accuracy: 0.4414 - val_loss: 1.0355 - val_accuracy: 0.4328
Epoch 5/50
170/170 [==============================] - 1s 4ms/step - loss: 1.0352 - accuracy: 0.4383 - val_loss: 1.0292 - val_accuracy: 0.4372
Epoch 6/50
170/170 [==============================] - 1s 3ms/step - loss: 1.0305 - accuracy: 0.4411 - val_loss: 1.0245 - val_accuracy: 0.4365
Epoch 7/50
170/170 [==============================] - 1s 4ms/step - loss: 1.0272 - accuracy: 0.4402 - val_loss: 1.0246 - val_accuracy: 0.4335
E

In [12]:
modelA.save("modelA.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
modelB , X_testB, y_testB = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.2,
                                         num_epochs=50)

Epoch 1/50
85/85 [==============================] - 2s 9ms/step - loss: 1.0766 - accuracy: 0.4208 - val_loss: 1.0695 - val_accuracy: 0.3870
Epoch 2/50
85/85 [==============================] - 0s 4ms/step - loss: 1.0518 - accuracy: 0.4226 - val_loss: 1.0596 - val_accuracy: 0.4357
Epoch 3/50
85/85 [==============================] - 0s 5ms/step - loss: 1.0382 - accuracy: 0.4440 - val_loss: 1.0589 - val_accuracy: 0.4328
Epoch 4/50
85/85 [==============================] - 0s 4ms/step - loss: 1.0293 - accuracy: 0.4429 - val_loss: 1.0573 - val_accuracy: 0.4210
Epoch 5/50
85/85 [==============================] - 0s 4ms/step - loss: 1.0301 - accuracy: 0.4448 - val_loss: 1.0520 - val_accuracy: 0.4239
Epoch 6/50
85/85 [==============================] - 0s 4ms/step - loss: 1.0200 - accuracy: 0.4507 - val_loss: 1.0491 - val_accuracy: 0.4431
Epoch 7/50
85/85 [==============================] - 0s 4ms/step - loss: 1.0091 - accuracy: 0.4599 - val_loss: 1.0275 - val_accuracy: 0.4372
Epoch 8/50
85/85 [==

In [14]:
modelB.save("modelB.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
modelC , X_testC, y_testC = compileModel(data=df,
                                         timestep=20,
                                         test_size=0.2,
                                         num_epochs=50)

Epoch 1/50
43/43 [==============================] - 3s 15ms/step - loss: 1.0903 - accuracy: 0.3755 - val_loss: 1.0839 - val_accuracy: 0.4130
Epoch 2/50
43/43 [==============================] - 0s 6ms/step - loss: 1.0698 - accuracy: 0.4205 - val_loss: 1.0768 - val_accuracy: 0.4130
Epoch 3/50
43/43 [==============================] - 0s 6ms/step - loss: 1.0518 - accuracy: 0.4442 - val_loss: 1.0487 - val_accuracy: 0.4572
Epoch 4/50
43/43 [==============================] - 0s 6ms/step - loss: 1.0397 - accuracy: 0.4457 - val_loss: 1.0419 - val_accuracy: 0.4454
Epoch 5/50
43/43 [==============================] - 0s 6ms/step - loss: 1.0479 - accuracy: 0.4442 - val_loss: 1.0309 - val_accuracy: 0.4218
Epoch 6/50
43/43 [==============================] - 0s 7ms/step - loss: 1.0401 - accuracy: 0.4516 - val_loss: 1.0297 - val_accuracy: 0.4749
Epoch 7/50
43/43 [==============================] - 0s 6ms/step - loss: 1.0325 - accuracy: 0.4701 - val_loss: 1.0275 - val_accuracy: 0.4690
Epoch 8/50
43/43 [=

In [16]:
modelC.save("modelC.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Since Model B has best performance use that to experiamnet with num epochs

In [17]:
modelD , X_testD, y_testD = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.2,
                                         num_epochs=75)

Epoch 1/75
85/85 [==============================] - 2s 9ms/step - loss: 1.0755 - accuracy: 0.4067 - val_loss: 1.0629 - val_accuracy: 0.4269
Epoch 2/75
85/85 [==============================] - 0s 4ms/step - loss: 1.0489 - accuracy: 0.4256 - val_loss: 1.0546 - val_accuracy: 0.4446
Epoch 3/75
85/85 [==============================] - 0s 4ms/step - loss: 1.0360 - accuracy: 0.4540 - val_loss: 1.0485 - val_accuracy: 0.4490
Epoch 4/75
85/85 [==============================] - 0s 4ms/step - loss: 1.0264 - accuracy: 0.4514 - val_loss: 1.0599 - val_accuracy: 0.4284
Epoch 5/75
85/85 [==============================] - 0s 4ms/step - loss: 1.0301 - accuracy: 0.4492 - val_loss: 1.0511 - val_accuracy: 0.4417
Epoch 6/75
85/85 [==============================] - 0s 5ms/step - loss: 1.0245 - accuracy: 0.4485 - val_loss: 1.0448 - val_accuracy: 0.4269
Epoch 7/75
85/85 [==============================] - 0s 4ms/step - loss: 1.0138 - accuracy: 0.4607 - val_loss: 1.0376 - val_accuracy: 0.4476
Epoch 8/75
85/85 [==

In [18]:
modelD.save("modelD.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
modelE , X_testE, y_testE = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.2,
                                         num_epochs=100)

Epoch 1/100
85/85 [==============================] - 2s 9ms/step - loss: 1.0756 - accuracy: 0.4100 - val_loss: 1.0646 - val_accuracy: 0.3944
Epoch 2/100
85/85 [==============================] - 0s 5ms/step - loss: 1.0503 - accuracy: 0.4333 - val_loss: 1.0567 - val_accuracy: 0.4461
Epoch 3/100
85/85 [==============================] - 0s 4ms/step - loss: 1.0375 - accuracy: 0.4440 - val_loss: 1.0509 - val_accuracy: 0.4638
Epoch 4/100
85/85 [==============================] - 0s 4ms/step - loss: 1.0263 - accuracy: 0.4499 - val_loss: 1.0596 - val_accuracy: 0.4328
Epoch 5/100
85/85 [==============================] - 0s 4ms/step - loss: 1.0306 - accuracy: 0.4403 - val_loss: 1.0510 - val_accuracy: 0.4284
Epoch 6/100
85/85 [==============================] - 0s 4ms/step - loss: 1.0227 - accuracy: 0.4459 - val_loss: 1.0459 - val_accuracy: 0.4313
Epoch 7/100
85/85 [==============================] - 0s 4ms/step - loss: 1.0129 - accuracy: 0.4603 - val_loss: 1.0373 - val_accuracy: 0.4490
Epoch 8/100
8

In [20]:
modelE.save("modelE.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Use best performance and experiment with test size

### 70-30 split

In [21]:
modelF , X_testF, y_testF = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.3,
                                         num_epochs=100)

Epoch 1/100
74/74 [==============================] - 2s 9ms/step - loss: 1.0734 - accuracy: 0.4054 - val_loss: 1.0965 - val_accuracy: 0.3811
Epoch 2/100
74/74 [==============================] - 0s 3ms/step - loss: 1.0521 - accuracy: 0.4253 - val_loss: 1.0791 - val_accuracy: 0.4216
Epoch 3/100
74/74 [==============================] - 0s 4ms/step - loss: 1.0393 - accuracy: 0.4269 - val_loss: 1.0725 - val_accuracy: 0.4047
Epoch 4/100
74/74 [==============================] - 0s 4ms/step - loss: 1.0310 - accuracy: 0.4362 - val_loss: 1.0680 - val_accuracy: 0.4435
Epoch 5/100
74/74 [==============================] - 0s 4ms/step - loss: 1.0283 - accuracy: 0.4367 - val_loss: 1.0618 - val_accuracy: 0.4604
Epoch 6/100
74/74 [==============================] - 0s 5ms/step - loss: 1.0257 - accuracy: 0.4464 - val_loss: 1.0665 - val_accuracy: 0.4519
Epoch 7/100
74/74 [==============================] - 0s 4ms/step - loss: 1.0248 - accuracy: 0.4535 - val_loss: 1.0490 - val_accuracy: 0.4772
Epoch 8/100
7

In [22]:
modelF.save("modelF.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### 50-50 split

In [23]:
modelG , X_testG, y_testG = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.5,
                                         num_epochs=100)

Epoch 1/100
53/53 [==============================] - 2s 12ms/step - loss: 1.0896 - accuracy: 0.4007 - val_loss: 1.0882 - val_accuracy: 0.3759
Epoch 2/100
53/53 [==============================] - 0s 4ms/step - loss: 1.0705 - accuracy: 0.4238 - val_loss: 1.0753 - val_accuracy: 0.4255
Epoch 3/100
53/53 [==============================] - 0s 4ms/step - loss: 1.0544 - accuracy: 0.4368 - val_loss: 1.0571 - val_accuracy: 0.4350
Epoch 4/100
53/53 [==============================] - 0s 4ms/step - loss: 1.0548 - accuracy: 0.4303 - val_loss: 1.0589 - val_accuracy: 0.4657
Epoch 5/100
53/53 [==============================] - 0s 4ms/step - loss: 1.0444 - accuracy: 0.4368 - val_loss: 1.0503 - val_accuracy: 0.4492
Epoch 6/100
53/53 [==============================] - 0s 4ms/step - loss: 1.0408 - accuracy: 0.4415 - val_loss: 1.0413 - val_accuracy: 0.4421
Epoch 7/100
53/53 [==============================] - 0s 4ms/step - loss: 1.0358 - accuracy: 0.4574 - val_loss: 1.0634 - val_accuracy: 0.4113
Epoch 8/100


In [24]:
modelG.save("modelG.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score,precision_recall_fscore_support

def getMetrics(model,X_test,y_test):

    y_pred_proba = model.predict(X_test)
    y_pred = np.argmax(y_pred_proba, axis=1)
    cm = confusion_matrix(y_test.flatten(), y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=[0,1,2], yticklabels=[0,1,2])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

    # Compute precision, recall, and F1 score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(y_test.flatten(), y_pred, average=None)

    # Print precision, recall, and F1 score for each class
    for i in range(len(precision)):
        print(f"Class {i}:")
        print(f"  Precision: {precision[i]}")
        print(f"  Recall: {recall[i]}")
        print(f"  F1 Score: {f1[i]}")
